# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import os

from sklearn.model_selection import train_test_split

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

## Setting up the data paths

In [4]:
data_dir = "C:/Users/krish/Downloads/spMohanty PlantVillage-Dataset master data_distribution_for_SVM"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)

## Importing the images in 224X224 Shape and 32 batch

In [20]:
batch_size = 32
img_height = 224
img_width = 224

#Image augmentation
train_datagen= ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=False,vertical_flip=False
                                  ,fill_mode='nearest',width_shift_range=0.2,height_shift_range=0.2)

val_datagen=ImageDataGenerator()


train_ds= train_datagen.flow_from_directory(directory=train_dir,batch_size=batch_size,target_size=(img_height,img_width),
                                         color_mode='rgb',class_mode='categorical',seed=369)

val_ds=val_datagen.flow_from_directory(directory=valid_dir,batch_size=32,target_size=(img_height,img_width),color_mode='rgb',class_mode='categorical')


TypeError: flow_from_directory() got an unexpected keyword argument 'rescale'

## Models Compilation and Training

### Importing ResNet50

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [6]:
# Setting up the base model
base_model_tf=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=38)

## Customizing the model for Our Input

In [7]:
#Model building
base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tf.cast(pt,tf.float32)
x=preprocess_input(func) #This function used to zero-center each color channel wrt Imagenet dataset
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=Dense(38,activation='softmax')(model_resnet)


model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272

## Setting up Callbacks for model fiting

In [8]:
#CallBacks
es=EarlyStopping(monitor='val_accuracy',verbose=1,patience=7,mode='auto') # It will stop the training of the model when it encounters the best weights
mc=ModelCheckpoint(filepath='/content',monitor='val_accuracy',verbose=1,save_best_only=True) # Creates Model checkpoints regularly
lr=ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=5,min_lr=0.001) # Decreseas the Learning Rate during the training

### Compiling the model

In [9]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Fiting or Training the Resnet50 Model

In [10]:
tf.config.run_functions_eagerly(True)

#Training
model_main.fit(train_ds,validation_data=val_ds,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

NameError: name 'train_ds' is not defined

In [ ]:
# Saving the model after training
model_main.save("RESNET50_PLANT_DISEASE.h5")

## Loading MobileNetV2 as Base Model

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

### Customizing the MobileNetV2 model for our input compiling the MobileNetV2 model and fiting or training the MobileNetV2 model

In [ ]:
# Setting up the inputs
inputs = pretrained_model.input

# Making a Neural net for using the pretrained model and matching the input and output layer of the pretrained model with our data
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(38, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


# Compiling the Custom Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Finally Training the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

# Results

## ResNet50 Results

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['loss'],color='b',label='Training loss')
plt.plot(model_main.history.history['val_loss'],color='r',label='Validation loss')
plt.xlabel("epochs")
plt.ylabel("loss_value")
plt.title("loss")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(model_main.history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("accuracy graph")

## MobileNetV2 Results

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'],color='b',label='Training loss')
plt.plot(history.history['val_loss'],color='r',label='Validation loss')
plt.xlabel("epochs")
plt.ylabel("loss_value")
plt.title("loss")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("accuracy graph")